In [1]:
import Pkg
Pkg.activate(".")

  Activating project at `~/Developer/DistributedStreams`


In [3]:
using Distributed

In [4]:
addprocs(2)

2-element Vector{Int64}:
 2
 3

In [5]:
@everywhere using DistributedStreams

In [6]:
input, output, control, status = launch_monitor(
    x->begin
        id = x.id
        data = deserialize(x.data) + 1
        Entry(id=x.id, data=serialize(data), valid=true)
    end;
    start_safe = false,
    verbose = true
)

(RemoteChannel{Channel{Entry}}(1, 1, 8), RemoteChannel{Channel{Entry}}(1, 1, 9), @NamedTuple{worker::Int64, safe::Base.RefValue{Bool}, flag::Base.RefValue{Bool}}[(worker = 2, safe = Base.RefValue{Bool}(false), flag = Base.RefValue{Bool}(false)), (worker = 3, safe = Base.RefValue{Bool}(false), flag = Base.RefValue{Bool}(false))], Any[Future(2, 1, 16, ReentrantLock(nothing, 0x00000000, 0x00, Base.GenericCondition{Base.Threads.SpinLock}(Base.IntrusiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), (6, 70598553600, 139630117278168)), nothing), Future(3, 1, 17, ReentrantLock(nothing, 0x00000000, 0x00, Base.GenericCondition{Base.Threads.SpinLock}(Base.IntrusiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), (2, 139629017495984, 139629962508624)), nothing)])

In [7]:
put!(input, Entry(id=0, data=serialize("hi there"), valid=false))

RemoteChannel{Channel{Entry}}(1, 1, 8)

In [10]:
typeof(status[1])

Future

In [12]:
t = @async fetch(status[1])

Task (runnable) @0x00007efe2b54c970

In [15]:
istaskdone(t)

true

In [21]:
t.result

RemoteException(2, CapturedException(MethodError(+, ("hi there", 1), 0x0000000000007ae8), Any[(#1 at In[6]:4, 1), (macro expansion at task.jl:479 [inlined], 1), (remote_worker at DistributedStreams.jl:182, 1), (#invokelatest#2 at essentials.jl:887, 1), (invokelatest at essentials.jl:884, 1), (#107 at process_messages.jl:282, 1), (run_work_thunk at process_messages.jl:70, 1), (run_work_thunk at process_messages.jl:79, 1), (#100 at process_messages.jl:88, 1)]))

In [22]:
typeof(t.result)

RemoteException

In [16]:
typeof(fetch(t))

LoadError: TaskFailedException

[91m    nested task error: [39mOn worker 2:
    MethodError: no method matching +(::String, ::Int64)
    
    [0mClosest candidates are:
    [0m  +(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m)
    [0m[90m   @[39m [90mBase[39m [90m[4moperators.jl:587[24m[39m
    [0m  +([91m::Base.CoreLogging.LogLevel[39m, ::Integer)
    [0m[90m   @[39m [90mBase[39m [90m[4mlogging.jl:131[24m[39m
    [0m  +([91m::Complex{Bool}[39m, ::Real)
    [0m[90m   @[39m [90mBase[39m [90m[4mcomplex.jl:320[24m[39m
    [0m  ...
    
    Stacktrace:
     [1] [0m[1m#1[22m
    [90m   @[39m [90m./[39m[90m[4mIn[6]:4[24m[39m
     [2] [0m[1mmacro expansion[22m
    [90m   @[39m [90m./[39m[90m[4mtask.jl:479[24m[39m[90m [inlined][39m
     [3] [0m[1mremote_worker[22m
    [90m   @[39m [90m~/Developer/DistributedStreams/src/[39m[90m[4mDistributedStreams.jl:182[24m[39m
     [4] [0m[1m#invokelatest#2[22m
    [90m   @[39